In [1]:
import pandas as pd
import os
from functools import partial

from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pickle
import numpy as np


In [233]:
with open('data/reg_set.p', 'rb') as handle:
    reg_set1 = pickle.load(handle)

final_reg = reg_set1[reg_set1['Year'] != 2022]
final_pred = reg_set1[reg_set1['Year'] == 2022]

exclude1415 = True
if exclude1415:
    final_reg = final_reg[final_reg['Year'] > 2015].copy()

In [234]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

reg_train, reg_test = train_test_split(final_reg, test_size = 0.25)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
base_vars = ['AverageDraftPositionPPR',
            'AverageDraftPositionPPRSq',
            'QB','RB','TE','WR']
base_x = (reg_train[base_vars].copy(), reg_test[base_vars].copy())

x_vars = ['Year',
            'AverageDraftPositionPPR',
            'AverageDraftPositionPPRSq',
            'foundAdp',
            'PrvPts_PPR',
            'foundLastYearStats',
            'qbDiff',
            'Draft',
            'Age',
            'PrvYrTmPts',
            'PlayersAtPosition',
            'PrvYrPtsShare',
            'QB','RB','TE','WR']
x = (reg_train[x_vars].copy(), reg_test[x_vars].copy())


y_pts = (reg_train['Pts_PPR'].copy(), reg_test['Pts_PPR'].copy())
y_cl = (reg_train['aboveBase'].copy(), reg_test['aboveBase'].copy())

def encodeYears(x):
    encoder = OneHotEncoder()
    y = x.copy()
    yrNames = [str(i) for i in list(y['Year'].unique())]
    yrNames.sort()
    y[yrNames] = encoder.fit_transform(y[['Year']]).toarray()
    return y.drop('Year', axis = 1)


x_0 = tuple([i.drop('Year', axis = 1) for i in list(x)])
x_a = tuple([encodeYears(i) for i in list(x)])

# x_b1, y_pts_b1, y_cl_b1 = 
# x_b2 = encodeYears(x_b1)

In [238]:
from sklearn.linear_model import LassoCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler

def try_lassos(X, y, polys = 2):
    X_train, X_test  = X
    y_train, y_test = y
    X_train = X_train.reset_index().drop('index', axis = 1)
    y_train = np.array(y_train, dtype = float)
    model = make_pipeline(PolynomialFeatures(polys), MaxAbsScaler(), LassoCV(cv = 5, random_state = 0, max_iter = 500000))
    model.fit(X_train, y_train)
    print(f"R^2 score on training data {model.score(X_train, y_train)}")
    print(f"R^2 score on test data {model.score(X_test, y_test)}")
    print()

try_lassos(x_0, y_pts, polys = 2)
try_lassos(x_0, y_pts, polys = 1)
try_lassos(x_a, y_pts, polys = 2)
try_lassos(x_a, y_pts, polys = 1)
try_lassos(base_x, y_pts, polys = 3)
try_lassos(base_x, y_pts, polys = 2)
try_lassos(base_x, y_pts, polys = 1)

R^2 score on training data 0.6772209996275458
R^2 score on test data 0.6758999383400408

R^2 score on training data 0.6410903066902294
R^2 score on test data 0.6402591490047865

R^2 score on training data 0.6654961607505775
R^2 score on test data 0.6695433280232002

R^2 score on training data 0.6416514361064506
R^2 score on test data 0.6410422498280499

R^2 score on training data 0.5643274878356084
R^2 score on test data 0.5940493833737095

R^2 score on training data 0.5586703086413507
R^2 score on test data 0.5841913093612976

R^2 score on training data 0.5090770553398156
R^2 score on test data 0.5213845218539082



In [75]:
print(x_a.shape)
print(x_0.shape)

(5724, 15)
(5724, 15)


In [304]:

from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor

def try_lassos(x, y, model_type, scorer, alphas = [0.005, 0.01, 0.025, 0.05, 0.1, 1, 5, 10], estimators = None, **kwargs):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

    alpha_scores = {}
    score = make_scorer(scorer)
    if estimators is None:
        estimators = [0]
    for alpha in alphas:
        for estimator_no in estimators:
            kwargs = {'alpha' : alpha, 'max_iter' : 500000}
            
            print("Cross validation scores...")
            score_arr = cross_val_score(model, X_train, y_train, cv = 5)#, scoring = score)
            avg_score = sum(score_arr) / len(score_arr)
            alpha_scores[(estimator_no, alpha)] = avg_score
    
    best_params = max(alpha_scores, key = alpha_scores.get)
    best_estimator_no, best_alpha = best_params
    print(alpha_scores)
    print(f"Best model had alpha of {best_params}. Now fitting model with this alpha on all training data...")
    model = make_pipeline(PolynomialFeatures(2), MaxAbsScaler(),  model_type(C = best_alpha
                                                                                ,max_iter = 500000
                                                                                # n_estimators = best_estimator_no
                                                                                ))
    model.fit(X_train, y_train)

    # print("Scoring model on test data...")
    final_score = model.score(X_test, y_test)
    # final_score = score(model, X_test, y_test)
    print(final_score)
    model.fit(x, y)
    return model
    
model_ridge = linear_model.Ridge
model_cls = linear_model.RidgeClassifier
model_log = linear_model.LogisticRegression
model_rf = RandomForestClassifier
model_gb = GradientBoostingRegressor

# best_model_pts = try_lassos(x, y_pts, model_ridge, r2_score, alphas = [.025, .05, 0.1, 0.25, 0.5, 1, 2.5, 5, 7.5, 10])
# best_model_log = try_lassos(x, y_cl, model_log, log_loss)
# best_model_pts = try_lassos(x, y_pts, model_gb, r2_score, alphas = [0.025, 0.05, 0.075, 0.1], estimators = [50, 100, 150, 200, 250])
# best_model_vbd = try_lassos(x, y_vbd, model, r2_score)
# best_model_cls = try_lassos(x, y_cl, model_rf, log_loss)

Cross validation scores...
Cross validation scores...
Cross validation scores...
Cross validation scores...
Cross validation scores...
Cross validation scores...
Cross validation scores...
Cross validation scores...
{(0, 0.005): 0.916049599916595, (0, 0.01): 0.9234491881043605, (0, 0.025): 0.9285541611280529, (0, 0.05): 0.927789832408059, (0, 0.1): 0.9270238746839732, (0, 1): 0.927279302525608, (0, 5): 0.9275334271639691, (0, 10): 0.9262572653582506}
Best model had alpha of (0, 0.025). Now fitting model with this alpha on all training data...
0.9224489795918367


In [152]:
def prepare_draft_template():
    # pred_x = pred_set[['AverageDraftPositionPPR', 'AverageDraftPositionPPRSq', 'foundLastYearStats', 'foundAdp', 'QB','RB','TE','WR']]
    a = np.column_stack((np.arange(1, 201), np.square(np.arange(1, 201)), np.ones(200), np.ones(200), 
                            np.ones(200), np.zeros(200), np.zeros(200), np.zeros(200)))
    arrs = [a]
    for i in range(5, 8):
        tmp = arrs[-1].copy()
        tmp[:, [i-1, i]] = tmp[:, [i, i - 1]]
        arrs.append(tmp)
    return arrs

arrs = prepare_draft_template()
final = pd.DataFrame(np.concatenate(arrs))
print(final)

         0        1    2    3    4    5    6    7
0      1.0      1.0  1.0  1.0  1.0  0.0  0.0  0.0
1      2.0      4.0  1.0  1.0  1.0  0.0  0.0  0.0
2      3.0      9.0  1.0  1.0  1.0  0.0  0.0  0.0
3      4.0     16.0  1.0  1.0  1.0  0.0  0.0  0.0
4      5.0     25.0  1.0  1.0  1.0  0.0  0.0  0.0
..     ...      ...  ...  ...  ...  ...  ...  ...
795  196.0  38416.0  1.0  1.0  0.0  0.0  0.0  1.0
796  197.0  38809.0  1.0  1.0  0.0  0.0  0.0  1.0
797  198.0  39204.0  1.0  1.0  0.0  0.0  0.0  1.0
798  199.0  39601.0  1.0  1.0  0.0  0.0  0.0  1.0
799  200.0  40000.0  1.0  1.0  0.0  0.0  0.0  1.0

[800 rows x 8 columns]


In [153]:
np_res = np.column_stack((best_model_pts.predict(final) , [i[1] for i in best_model_log.predict_proba(final)]))
final_additions = pd.DataFrame(np_res)
final_additions.columns = ['Predicted Points', 'Predicted Prob']
final = final.join(final_additions)
final.to_excel('draftPositionPredictions2.xlsx')
# final[final[0].isin([1, 10, 20, 30, 40, 50, 60, 70, 80])]

In [305]:
final_pred = final_pred.reset_index().drop('index', axis = 1)
final_pred['Preds'] = best_model_pts.predict(pred_x)
final_pred['Prob'] = [i[1] for i in best_model_log.predict_proba(pred_x)]
final_pred.sort_values('AverageDraftPositionPPR', ascending = True, inplace = True)
print(final_pred.head())
final_pred.to_excel('test_4.xlsx')

                  Player   Tm FantPos  Year  PrvPts_PPR  PrvMyVBD  \
351      Jonathan Taylor  IND      RB  2022       373.1     187.5   
435  Christian McCaffrey  CAR      RB  2022       127.5     -58.1   
21         Austin Ekeler  LAC      RB  2022       343.8     158.2   
209        Derrick Henry  TEN      RB  2022       193.3       7.7   
200     Justin Jefferson  MIN      WR  2022       330.4     127.9   

     foundLastYearStats  AverageDraftPositionPPR  foundAdp  QB  ...  \
351                   1                      1.3         1   0  ...   
435                   1                      2.4         1   0  ...   
21                    1                      2.7         1   0  ...   
209                   1                      4.3         1   0  ...   
200                   1                      4.6         1   0  ...   

           DOB  Draft        Age  ones  PrvYrTmPts  PlayersAtPosition  \
351 1999-01-19     41  23.632877     1       582.1                  5   
435 1996-06-

In [1983]:
pred_set

,Player,Tm,FantPos,Year,PrvPts_HPPR,PrvMyVBD,foundLastYearStats,AverageDraftPositionPPR,foundAdp,QB,RB,TE,WR,Pts_HPPR,MyVBD,Base,AverageDraftPositionPPRSq
6531,Josh Allen,BUF,QB,2022,402.58,102.100000,1,20.4,1,1,0,0,0,NaN,NaN,NaN,416.16
6532,Justin Herbert,LAC,QB,2022,380.76,80.280000,1,33.2,1,1,0,0,0,NaN,NaN,NaN,1102.24
6533,Tom Brady,TAM,QB,2022,374.74,74.260000,1,86.8,1,1,0,0,0,NaN,NaN,NaN,7534.24
6534,Patrick Mahomes,KAN,QB,2022,361.66,61.180000,1,38.3,1,1,0,0,0,NaN,NaN,NaN,1466.89
6535,Aaron Rodgers,GNB,QB,2022,332.80,32.320000,1,80.2,1,1,0,0,0,NaN,NaN,NaN,6432.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7411,Tre Turner,LVR,WR,2022,0.00,-100.954626,0,173.0,0,0,0,0,1,NaN,NaN,NaN,29929.00
7412,Britain Covey,PHI,WR,2022,0.00,-100.954626,0,173.0,0,0,0,0,1,NaN,NaN,NaN,29929.00
7413,Jerrion Ealy,KAN,WR,2022,0.00,-100.954626,0,173.0,0,0,0,0,1,NaN,NaN,NaN,29929.00
7414,Makai Polk,BAL,WR,2022,0.00,-100.954626,0,173.0,0,0,0,0,1,NaN,NaN,NaN,29929.00


In [469]:
np.arange(0, 100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [90]:
x

,PrvMyVBD,foundLastYear,AverageDraftPositionPPR,foundAdp,QB,RB,TE,WR
0,-91.480000,1,20.8,1,1,0,0,0
1,31.660000,1,51.9,1,1,0,0,0
2,9.260000,1,104.2,1,1,0,0,0
3,149.060000,1,9.4,1,1,0,0,0
4,-2.080000,1,129.5,1,1,0,0,0
...,...,...,...,...,...,...,...,...
4419,-157.600000,1,201.0,0,0,0,0,1
4420,-107.454626,0,201.0,0,0,0,0,1
4421,-107.454626,0,201.0,0,0,0,0,1
4422,-107.454626,0,201.0,0,0,0,0,1


In [89]:
best_model['lasso'].coef_
# best_model['polynomialfeatures'].feature_names_in_

array([ 0.00000000e+00,  2.54886880e+02, -1.50302903e-01, -2.74045207e+02,
        2.40678061e-02, -1.09929976e+01,  3.00549425e-04, -0.00000000e+00,
        0.00000000e+00, -7.33820953e+01, -1.54424049e+02, -2.12188648e+01,
       -7.20965040e+01, -1.31897977e+01,  3.04093150e+00,  7.32278857e+01,
       -0.00000000e+00, -5.79969974e+01,  4.09516946e+01, -8.35034331e+00,
       -2.08291578e+01, -0.00000000e+00,  4.75539772e+00, -5.83062617e+00,
        1.15422776e+02,  0.00000000e+00, -2.35040264e+01, -1.03075303e+01,
        8.29136808e+01,  0.00000000e+00,  2.45260797e+01,  5.80305935e+01,
       -1.67563932e+01,  2.21129756e+01, -0.00000000e+00, -5.85855438e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.30796065e+01,
        0.00000000e+00,  0.00000000e+00, -4.00616248e+01,  0.00000000e+00,
        6.23331578e+00])

In [411]:
test.groupby(['FantPos', 'Year'], as_index = False).mean()[['FantPos','Year', 'MyVBD']]


,FantPos,Year,MyVBD
0,QB,2014,-159.503733
1,QB,2015,-167.549167
2,QB,2016,-145.828824
3,QB,2017,-151.290411
4,QB,2018,-166.008649
5,QB,2019,-153.993239
6,QB,2020,-217.272683
7,QB,2021,-189.569880
8,RB,2014,-67.214286
9,RB,2015,-73.550675


In [317]:
test = adp_df.copy()
test['New Name'] = test['Name'].str.replace('([I ]+$)', '',regex= True)
test[test['Name'] != test['New Name']]

,Name,Year,Team,Position,PositionRank,AverageDraftPositionPPR,CB,DB,DE,DL,...,LS,NT,OLB,QB,RB,S,SS,TE,WR,New Name


In [189]:
pts_df

,Player,Tm,FantPos,Age,G,GS,PassCmp,PassAtt,PassYds,PassTD,...,VBD,PosRank,OvRank,Year,Pts_HPPR,Pts_PPR,MyRk,Baseline,Base,MyVBD
0,Aaron Rodgers,GNB,QB,31,16,16,341,520,4381.0,38.0,...,89,1,13,2014,354.14,354.14,1.0,0,269.5,84.64
1,Andrew Luck,IND,QB,25,16,16,380,616,4761.0,40.0,...,86,2,14,2014,351.74,351.74,2.0,0,269.5,82.24
2,Russell Wilson,SEA,QB,26,16,16,285,452,3475.0,20.0,...,64,3,22,2014,330.10,330.60,3.0,0,269.5,61.10
3,Peyton Manning,DEN,QB,38,16,16,395,597,4727.0,39.0,...,47,4,32,2014,312.68,312.68,4.0,0,269.5,43.18
4,Ben Roethlisberger,PIT,QB,32,16,16,408,608,4952.0,32.0,...,41,5,37,2014,306.18,306.18,5.0,0,269.5,36.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4917,Justin Watson,TAM,NaN,25,1,0,0,0,0.0,0.0,...,NaN,255,NaN,2021,0.00,0.00,478.0,0,NaN,NaN
4918,Nsimba Webster,CHI,NaN,25,6,0,0,0,0.0,0.0,...,NaN,256,NaN,2021,0.00,0.00,479.0,0,NaN,NaN
4919,David Wells,ARI,NaN,26,3,1,0,0,0.0,0.0,...,NaN,142,NaN,2021,0.00,0.00,480.0,0,NaN,NaN
4920,Jordan Wilkins,2TM,NaN,27,5,0,0,0,0.0,0.0,...,NaN,176,NaN,2021,0.00,0.00,481.0,0,NaN,NaN


In [188]:
adp_df

,Name,Year,Team,Position,PositionRank,AverageDraftPositionPPR,CB,DB,DE,DL,...,LB,LS,NT,OLB,QB,RB,S,SS,TE,WR
0,LeSean McCoy,2014,PHI,RB,1,1.8,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Jamaal Charles,2014,KC,RB,2,2.7,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Adrian Peterson,2014,MIN,RB,3,3.9,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,Eddie Lacy,2014,GB,RB,4,5.9,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Matt Forte,2014,CHI,RB,5,6.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9742,Isaiah Dunn,2021,NYJ,CB,496,2111.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9743,Shaun Crawford,2021,LV,CB,497,2112.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9744,Barrington Wade,2021,DEN,ILB,353,2113.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9745,Jovan Swann,2021,BAL,DT,399,2114.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
d = best_model_log.predict_proba(x)
reg_set['probs'] = [d[1] for d in d]

In [ ]:
reg_set.sort_values(['Year','AverageDraftPositionPPR'], ascending = [False, True])

,Player,Tm,FantPos,Year,PrvMyVBD,foundLastYearStats,AverageDraftPositionPPR,foundAdp,QB,RB,TE,WR,Pts_HPPR,MyVBD,Base,AverageDraftPositionPPRSq,VBD_cls,probs
3991,Christian McCaffrey,CAR,RB,2021,-74.500000,1,1.3,1,0,1,0,0,109.00,-59.60,168.6,1.69,0,0.721385
3967,Dalvin Cook,MIN,RB,2021,159.400000,1,2.5,1,0,1,0,0,189.30,20.70,168.6,6.25,1,0.715512
3960,Alvin Kamara,NOR,RB,2021,179.900000,1,3.1,1,0,1,0,0,211.20,42.60,168.6,9.61,1,0.712551
4245,Davante Adams,GNB,WR,2021,138.200000,1,4.5,1,0,0,0,1,282.80,114.30,168.5,20.25,1,0.754620
3958,Ezekiel Elliott,DAL,RB,2021,41.300000,1,5.3,1,0,1,0,0,228.56,59.96,168.6,28.09,1,0.701557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,Micheal Spurlock,CHI,WR,2014,-87.233021,0,171.0,0,0,0,0,1,1.30,-164.20,165.5,29241.00,0,0.007729
524,Kevin Dorsey,GNB,WR,2014,-87.233021,0,171.0,0,0,0,0,1,0.90,-164.60,165.5,29241.00,0,0.007729
525,De'Andre Presley,CAR,WR,2014,-87.233021,0,171.0,0,0,0,0,1,-0.20,-165.70,165.5,29241.00,0,0.007729
526,Jalen Saunders,4TM,WR,2014,-87.233021,0,171.0,0,0,0,0,1,-0.80,-166.30,165.5,29241.00,0,0.007729


In [ ]:
def prepare_draft_template():
    pred_x = pred_set[['AverageDraftPositionPPR', 'AverageDraftPositionPPRSq', 'foundLastYearStats', 'foundAdp', 'QB','RB','TE','WR']]
    a = np.column_stack((np.arange(1, 201), np.square(np.arange(1, 201)), np.ones(200), np.ones(200), 
                            np.ones(200), np.zeros(200), np.zeros(200), np.zeros(200)))
    arrs = [a]
    for i in range(5, 8):
        tmp = arrs[-1].copy()
        tmp[:, [i-1, i]] = tmp[:, [i, i - 1]]
        arrs.append(tmp)
    return arrs

arrs = prepare_draft_template()
final = pd.DataFrame(np.concatenate(arrs))
print(final)

         0        1    2    3    4    5    6    7
0      1.0      1.0  1.0  1.0  1.0  0.0  0.0  0.0
1      2.0      4.0  1.0  1.0  1.0  0.0  0.0  0.0
2      3.0      9.0  1.0  1.0  1.0  0.0  0.0  0.0
3      4.0     16.0  1.0  1.0  1.0  0.0  0.0  0.0
4      5.0     25.0  1.0  1.0  1.0  0.0  0.0  0.0
..     ...      ...  ...  ...  ...  ...  ...  ...
795  196.0  38416.0  1.0  1.0  0.0  0.0  0.0  1.0
796  197.0  38809.0  1.0  1.0  0.0  0.0  0.0  1.0
797  198.0  39204.0  1.0  1.0  0.0  0.0  0.0  1.0
798  199.0  39601.0  1.0  1.0  0.0  0.0  0.0  1.0
799  200.0  40000.0  1.0  1.0  0.0  0.0  0.0  1.0

[800 rows x 8 columns]


In [2204]:
list(zip(best_model_pts['polynomialfeatures'].get_feature_names(x.columns), best_model_pts['ridge'].coef_))

[('1', 0.0),
 ('PrvMyVBD', 4.802426512055645),
 ('AverageDraftPositionPPR', -87.95756938449912),
 ('AverageDraftPositionPPRSq', 35.798974770178866),
 ('foundLastYearStats', 11.844969855555506),
 ('foundAdp', 15.053889123591345),
 ('QB', 9.780508926901687),
 ('RB', 13.903716581185568),
 ('TE', -23.30773944173522),
 ('WR', -0.3764860666669353),
 ('Draft', -108.75811088344895),
 ('Age', -153.56683680731044),
 ('PrvYrTmPts', -42.26613053434552),
 ('PlayersAtPosition', -39.49193679185889),
 ('OldQBs', 4.1388680937978535),
 ('NewQBs', -27.109242751358508),
 ('PrvMyVBD^2', -21.838160456116267),
 ('PrvMyVBD AverageDraftPositionPPR', -4.0238025504697115),
 ('PrvMyVBD AverageDraftPositionPPRSq', 20.8623197348867),
 ('PrvMyVBD foundLastYearStats', 98.7057181111844),
 ('PrvMyVBD foundAdp', -78.05223894441255),
 ('PrvMyVBD QB', -0.7875512357588327),
 ('PrvMyVBD RB', -6.86377005867569),
 ('PrvMyVBD TE', -3.039410419971752),
 ('PrvMyVBD WR', 24.156756359656182),
 ('PrvMyVBD Draft', 2.916624867231286)